In [1]:
import glob
import pandas as pd
import numpy as np
import keras 
import matplotlib.pylab as plt
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import keras 
import random
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Conv2DTranspose, MaxPooling2D, BatchNormalization, Activation, concatenate, Input, GlobalAveragePooling2D
from tensorflow.keras import Model
import warnings
import keras.backend as K 
from sklearn.model_selection import KFold
from keras.utils import Sequence
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold


warnings.filterwarnings("ignore")
np.random.seed(7)
random.seed(7)
tf.random.set_seed(7)


def chk(x):  
    grid = x[:,:,14]
    for i in range(40): 
        for j in range(40): 
            if grid[i][j] < 0: 
                return False 
    return True  

def balance(x):  
    return np.sum(x[:,:,14]) >= 50 

Using TensorFlow backend.


In [2]:
train_files = os.listdir('./storage/train_proc')
submission = pd.read_csv('./storage/sample_submission.csv')

train = []
cnt = 0 
for file in train_files: 
    data = np.load('./storage/train_proc/'+file).astype('float32') 
    train.append(data)
        
        
#test = []
#for sub_id in submission['id']:
#    data = np.load('./storage/test/'+'subset_'+sub_id+'.npy').astype('float32')
#    test.append(data)
train = np.array(train)
#test = np.array(test) 

print(train.shape) 
#print(test.shape)

(31406, 40, 40, 15)


In [3]:
x_train = train[:,:,:,:10]
y_train = train[:,:,:,14]
#test = test[:,:,:,:10] 

del train

In [4]:
def build_model(inputs):

    bn = BatchNormalization()(inputs)
    conv0 = Conv2D(256, kernel_size=1, strides=1, padding='same', activation='relu')(bn)

    bn = BatchNormalization()(conv0)
    conv = Conv2D(128, kernel_size=2, strides=1, padding='same', activation='relu')(bn)
    concat = concatenate([conv0, conv], axis=3)

    bn = BatchNormalization()(concat)
    conv = Conv2D(64, kernel_size=3, strides=1, padding='same', activation='relu')(bn)
    concat = concatenate([concat, conv], axis=3)

    for i in range(12):
        bn = BatchNormalization()(concat)
        conv = Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu')(bn)
        concat = concatenate([concat, conv], axis=3)

    bn = BatchNormalization()(concat)
    outputs = Conv2D(1, kernel_size=1, strides=1, padding='same', activation='relu')(bn)

    model = Model(inputs=inputs, outputs=outputs)

    return model

input_layer = Input((40, 40, 10))
model = build_model(input_layer)


In [5]:
from sklearn.metrics import f1_score
def custom_loss(y_true, y_pred): 
    y_true = K.flatten(y_true) 
    y_pred = K.flatten(y_pred) 
    MAE = K.mean(K.abs(y_true-y_pred))   
    
    y_true = keras.backend.less_equal(0.1,y_true)
    y_true = keras.backend.cast(y_true, dtype='float32')
    y_pred = keras.backend.less_equal(0.1,y_pred)
    y_pred = keras.backend.cast(y_pred, dtype='float32')
    
    dice = 2*K.sum(y_true * y_pred)/(K.sum(y_true) + K.sum(y_pred) + 1e-2)
    loss = MAE/dice 
    return loss


def dice_score(y_true, y_pred):    
    remove_NAs = y_true >= 0 
    y_true = y_true[remove_NAs] 
    y_pred = y_pred[remove_NAs] 
    return (2*K.sum(y_true * y_pred)/(K.sum(y_true) + K.sum(y_pred) + 1e-2)) 

def mae(y_true, y_pred) :
    
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    
    y_pred = y_pred.reshape(1, -1)[0]
    
    over_threshold = y_true >= 0.1
    
    return np.mean(np.abs(y_true[over_threshold] - y_pred[over_threshold]))
    

def fscore(y_true, y_pred):
    
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    
    y_pred = y_pred.reshape(1, -1)[0]
    
    remove_NAs = y_true >= 0
    
    y_true = np.where(y_true[remove_NAs] >= 0.1, 1, 0)
    
    y_pred = np.where(y_pred[remove_NAs] >= 0.1, 1, 0)
    
    return(f1_score(y_true, y_pred))

def maeOverFscore(y_true, y_pred):
    
    return mae(y_true, y_pred) / (fscore(y_true, y_pred) + 1e-07)

def mae_keras(y_true, y_pred):  
    score = tf.py_function(func=mae, inp=[y_true,y_pred], Tout=tf.float32, name = 'mae_keras') 
    return score   

def dice_score_keras(y_true, y_pred): 
    score = tf.py_function(func=dice_score, inp=[y_true,y_pred],Tout=tf.float32,name = 'dice_score_keras')
    return score 


def fscore_keras(y_true, y_pred):
    score = tf.py_function(func=fscore, inp=[y_true, y_pred], Tout=tf.float32, name='fscore_keras')
    return score

def maeOverFscore_keras(y_true, y_pred):
    score = tf.py_function(func=maeOverFscore, inp=[y_true, y_pred], Tout=tf.float32,  name='custom_mse') 
    return score


In [ ]:
def k_fold(k,files):  
    folds = [] 
    fold_size = len(files) // k 
    for i in range(k): 
        if i == k-1:  
            l = files[i*fold_size:] 
        else: 
            l = files[i*fold_size:(i+1)*fold_size]  
        folds.append(l)   
    return folds  

def k_fold_train(k,x_train,y_train):  
    x_folds = k_fold(k,x_train) 
    y_folds = k_fold(k,y_train) 
    for t in range(k):  
        print("************ Fold {} training ************".format(t+1)) 
        if t == 0: 
            continue # use the second fold 
        cur_val_x = x_folds[t] 
        cur_val_y = y_folds[t] 
        train_folds_x = x_folds[0:t] + x_folds[t+1:] 
        train_folds_y = y_folds[0:t] + y_folds[t+1:]  
        cur_train_x = [] 
        cur_train_y = [] 
        for j in train_folds_x:  
            for q in j:  
                cur_train_x.append(q) 
        for j in train_folds_y:  
            for q in j:  
                cur_train_y.append(q)  

                
        cur_train_x = np.asarray(cur_train_x) 
        cur_train_y = np.asarray(cur_train_y) 
        cur_train_x = cur_train_x.reshape((-1,40,40,10)) 
        cur_train_y = cur_train_y.reshape((-1,40,40,1)) 
        
        cur_val_x = np.asarray(cur_val_x) 
        cur_val_y = np.asarray(cur_val_y) 
        cur_val_x = cur_val_x.reshape((-1,40,40,10)) 
        cur_val_y = cur_val_y.reshape((-1,40,40,1)) 
        
        # augment data for cur_train_x and cur_train_y 
        print("rotate 90...")
        rotate_X_90 = np.zeros_like(cur_train_x)
        rotate_Y_90 = np.zeros_like(cur_train_y)  
        for j in range(rotate_X_90.shape[0]):
            rotate_x=np.zeros([cur_train_x.shape[1],cur_train_x.shape[2],10])
            rotate_y=np.zeros([cur_train_x.shape[1],cur_train_x.shape[2],1])
            for i in range(10):
                rotate_x[:,:,i]=np.rot90(cur_train_x[j,:,:,i])
            rotate_y[:,:,0]=np.rot90(cur_train_y[j,:,:,0])
            rotate_X_90[j,:,:,:] = rotate_x
            rotate_Y_90[j,:,:,:] = rotate_y
    
        print("rotate 270...")
        rotate_X_270 = np.zeros_like(cur_train_x)
        rotate_Y_270 = np.zeros_like(cur_train_y)
        for j in range(rotate_X_270.shape[0]):
            rotate_x=np.zeros([cur_train_x.shape[1],cur_train_x.shape[2],10])
            rotate_y=np.zeros([cur_train_x.shape[1],cur_train_x.shape[2],1])
            for i in range(10):
                rotate_x[:,:,i]=np.rot90(cur_train_x[j,:,:,i])
                rotate_x[:,:,i]=np.rot90(rotate_x[:,:,i])
                rotate_x[:,:,i]=np.rot90(rotate_x[:,:,i])
            rotate_y[:,:,0]=np.rot90(cur_train_y[j,:,:,0])
            rotate_y[:,:,0]=np.rot90(rotate_y[:,:,0])
            rotate_y[:,:,0]=np.rot90(rotate_y[:,:,0])
            rotate_X_270[j,:,:,:] = rotate_x
            rotate_Y_270[j,:,:,:] = rotate_y

        cur_train_x = np.concatenate((cur_train_x, rotate_X_90, rotate_X_270), axis = 0)
        cur_train_y = np.concatenate((cur_train_y, rotate_Y_90, rotate_Y_270), axis = 0)
        del rotate_X_90, rotate_Y_90, rotate_X_270, rotate_Y_270  
        
        print("Transpose...")
        x_T = np.zeros_like(cur_train_x)
        y_T = np.zeros_like(cur_train_y)

        for i in range(cur_train_x.shape[0]):
            for j in range(cur_train_x.shape[3]):
                x_T[i,:,:,j] = cur_train_x[i,:,:,j].T
            y_T[i,:,:,0] = cur_train_y[i,:,:,0].T

        cur_train_x = np.concatenate((cur_train_x, x_T), axis = 0)
        cur_train_y = np.concatenate((cur_train_y, y_T), axis = 0)

        del x_T,y_T
        
        print("train data shape...")
        print(cur_train_x.shape) 
        print(cur_train_y.shape)
        print("validation data shape...")
        print(cur_val_x.shape) 
        print(cur_val_y.shape) 
        # start training 
        input_layer = Input((40, 40, 10))
        model = build_model(input_layer)  
        model.compile(loss=custom_loss, optimizer="adam", metrics=[maeOverFscore_keras, fscore_keras, dice_score_keras, mae_keras])
        model_checkpoint = keras.callbacks.ModelCheckpoint('./storage/base_model4/kfold'+str(t+1)+'/model{epoch:02d}.h5',period=1)
        lr_adjust = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',patience=3,factor=0.8) 
        model_history = model.fit(cur_train_x, cur_train_y ,epochs = 100, callbacks = [model_checkpoint,lr_adjust], validation_data = (cur_val_x, cur_val_y), shuffle = True,verbose = 1)

k_fold_train(2,x_train,y_train) 
        

************ Fold 1 training ************
************ Fold 2 training ************
rotate 90...
rotate 270...
Transpose...
train data shape...
(94218, 40, 40, 10)
(94218, 40, 40, 1)
validation data shape...
(15703, 40, 40, 10)
(15703, 40, 40, 1)
Train on 94218 samples, validate on 15703 samples
Epoch 1/100
94218/94218 [==============================] - 640s 7ms/sample - loss: 0.3521 - maeOverFscore_keras: 2.1613 - fscore_keras: 0.7097 - dice_score_keras: 5.8020 - mae_keras: 1.5183 - val_loss: 0.2964 - val_maeOverFscore_keras: 1.8903 - val_fscore_keras: 0.7498 - val_dice_score_keras: 6.1779 - val_mae_keras: 1.4170
Epoch 2/100
94218/94218 [==============================] - 631s 7ms/sample - loss: 0.3057 - maeOverFscore_keras: 1.9259 - fscore_keras: 0.7405 - dice_score_keras: 7.3905 - mae_keras: 1.4250 - val_loss: 0.2846 - val_maeOverFscore_keras: 1.8134 - val_fscore_keras: 0.7579 - val_dice_score_keras: 6.4333 - val_mae_keras: 1.3748
Epoch 3/100
94218/94218 [============================

In [8]:
test_path = './storage/test'
test_files = sorted(glob.glob(test_path + '/*'))

X_test = []

for file in tqdm(test_files, desc = 'test'):
    
    data = np.load(file)
    
    X_test.append(data[:,:,:10])
                  
X_test = np.array(X_test)


test: 100%|██████████| 2416/2416 [00:11<00:00, 210.73it/s]


In [9]:
model = tf.keras.models.load_model('./storage/base_model4/kfold2/model46.h5',compile=False) 
model.compile(loss=custom_loss,optimizer='adam',metrics=[maeOverFscore_keras, fscore_keras, dice_score_keras, mae_keras])
print("predicting...")
pred = model.predict(X_test) 
print("making csv file...")
submission = pd.read_csv('./storage/sample_submission.csv') 
submission.iloc[:,1:] = pred.reshape(-1,1600) 
submission.to_csv('./storage/epoch46_m2.csv',index=False) 

predicting...
making csv file...


In [10]:
model = tf.keras.models.load_model('./storage/base_model4/kfold2/model52.h5',compile=False) 
model.compile(loss=custom_loss,optimizer='adam',metrics=[maeOverFscore_keras, fscore_keras, dice_score_keras, mae_keras])
print("predicting...")
pred = model.predict(X_test) 
print("making csv file...")
submission = pd.read_csv('./storage/sample_submission.csv') 
submission.iloc[:,1:] = pred.reshape(-1,1600)  
submission.to_csv('./storage/epoch52_m2.csv',index=False) 

predicting...
making csv file...


In [11]:
model = tf.keras.models.load_model('./storage/base_model4/kfold2/model53.h5',compile=False) 
model.compile(loss=custom_loss,optimizer='adam',metrics=[maeOverFscore_keras, fscore_keras, dice_score_keras, mae_keras])
print("predicting...")
pred = model.predict(X_test) 
print("making csv file...")
submission = pd.read_csv('./storage/sample_submission.csv') 
submission.iloc[:,1:] = pred.reshape(-1,1600)  
submission.to_csv('./storage/epoch53_m2.csv',index=False) 

predicting...
making csv file...


In [12]:
model = tf.keras.models.load_model('./storage/base_model4/kfold2/model58.h5',compile=False) 
model.compile(loss=custom_loss,optimizer='adam',metrics=[maeOverFscore_keras, fscore_keras, dice_score_keras, mae_keras])
print("predicting...")
pred = model.predict(X_test) 
print("making csv file...")
submission = pd.read_csv('./storage/sample_submission.csv') 
submission.iloc[:,1:] = pred.reshape(-1,1600)  
submission.to_csv('./storage/epoch58_m2.csv',index=False) 

predicting...
making csv file...


In [13]:
model = tf.keras.models.load_model('./storage/base_model4/kfold2/model67.h5',compile=False) 
model.compile(loss=custom_loss,optimizer='adam',metrics=[maeOverFscore_keras, fscore_keras, dice_score_keras, mae_keras])
print("predicting...")
pred = model.predict(X_test) 
print("making csv file...")
submission = pd.read_csv('./storage/sample_submission.csv') 
submission.iloc[:,1:] = pred.reshape(-1,1600)  
submission.to_csv('./storage/epoch67_m2.csv',index=False) 

predicting...
making csv file...


In [14]:
model = tf.keras.models.load_model('./storage/base_model4/kfold2/model74.h5',compile=False) 
model.compile(loss=custom_loss,optimizer='adam',metrics=[maeOverFscore_keras, fscore_keras, dice_score_keras, mae_keras])
print("predicting...")
pred = model.predict(X_test) 
print("making csv file...")
submission = pd.read_csv('./storage/sample_submission.csv') 
submission.iloc[:,1:] = pred.reshape(-1,1600)  
submission.to_csv('./storage/epoch74_m2.csv',index=False) 

predicting...
making csv file...


In [15]:
model = tf.keras.models.load_model('./storage/basenet/kfold3/model_98.h5',compile=False) 
model.compile(loss=custom_loss,optimizer='adam',metrics=[maeOverFscore_keras, fscore_keras, dice_score_keras, mae_keras])
print("predicting...")
pred = model.predict(X_test) 
print("making csv file...")
submission = pd.read_csv('./storage/sample_submission.csv') 
submission.iloc[:,1:] = pred.reshape(-1,1600)  
submission.to_csv('./storage/epoch98_kev.csv',index=False) 

predicting...
making csv file...


In [16]:
model = tf.keras.models.load_model('./storage/basenet/kfold3/model_92.h5',compile=False) 
model.compile(loss=custom_loss,optimizer='adam',metrics=[maeOverFscore_keras, fscore_keras, dice_score_keras, mae_keras])
print("predicting...")
pred = model.predict(X_test) 
print("making csv file...")
submission = pd.read_csv('./storage/sample_submission.csv') 
submission.iloc[:,1:] = pred.reshape(-1,1600)  
submission.to_csv('./storage/epoch92_kev.csv',index=False) 

predicting...
making csv file...


In [17]:
model = tf.keras.models.load_model('./storage/basenet/kfold3/model_80.h5',compile=False) 
model.compile(loss=custom_loss,optimizer='adam',metrics=[maeOverFscore_keras, fscore_keras, dice_score_keras, mae_keras])
print("predicting...")
pred = model.predict(X_test) 
print("making csv file...")
submission = pd.read_csv('./storage/sample_submission.csv') 
submission.iloc[:,1:] = pred.reshape(-1,1600)  
submission.to_csv('./storage/epoch80_kev.csv',index=False) 

predicting...
making csv file...


In [18]:
model = tf.keras.models.load_model('./storage/basenet/kfold3/model_89.h5',compile=False) 
model.compile(loss=custom_loss,optimizer='adam',metrics=[maeOverFscore_keras, fscore_keras, dice_score_keras, mae_keras])
print("predicting...")
pred = model.predict(X_test) 
print("making csv file...")
submission = pd.read_csv('./storage/sample_submission.csv') 
submission.iloc[:,1:] = pred.reshape(-1,1600)  
submission.to_csv('./storage/epoch89_kev.csv',index=False) 

predicting...
making csv file...
